In [1]:
import pandas as pd

In [2]:
# import csv file to dataframes
cars_df = pd.read_csv("Car_Detail.csv")
house_df = pd.read_csv("Median_Household_Incomes.csv")

In [3]:
# NA count for each column in cars_df
cars_df.isnull().sum()

Make                    0
Model                   0
Year                    0
Engine Fuel Type        3
Engine HP              69
Engine Cylinders       30
Transmission Type       0
Driven_Wheels           0
Number of Doors         6
Market Category      3742
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64

In [4]:
# Median MSRP of cars that are Convertibles
cars_df[cars_df["Vehicle Style"] == "Convertible"]["MSRP"].median()

44350.0

In [5]:
# Engine Horse Power of the cars with the least Populatiry
cars_df[cars_df["Popularity"] == cars_df["Popularity"].min()]["Engine HP"]

2116    400.0
2117    400.0
2118    400.0
Name: Engine HP, dtype: float64

In [6]:
# Extract year from date into Year column
house_df['Year'] = pd.DatetimeIndex(house_df['DATE']).year

# Find how many years house_df data spans
house_df["Year"].max() - house_df["Year"].min()

34

In [7]:
# Year of the Highest median income in Texas
house_df[house_df["Median_TX"] == house_df["Median_TX"].max()]["DATE"]

33    1/1/17
Name: DATE, dtype: object

In [8]:
# Remove all rows with NA from cars_df
cars_df_no_na = cars_df.dropna()

# Create Database with cars that only have all wheel drive
cars_allwheel = cars_df_no_na[cars_df_no_na["Driven_Wheels"] == "all wheel drive"]

# Top Three cars with all wheel drive by make
cars_allwheel.groupby(["Make"])["Driven_Wheels"].count().sort_values(ascending=False).head(3)

Make
Audi      254
Subaru    190
BMW       144
Name: Driven_Wheels, dtype: int64

In [9]:
# The top 5 Years with the lowest average Highway mpg
cars_df_no_na.groupby(["Year"])["highway MPG"].mean().sort_values().head(5)

Year
2007    21.163090
2003    21.761364
2002    21.915493
2008    22.478049
2004    22.637363
Name: highway MPG, dtype: float64

In [10]:
# drop all rows with NA in house database
house_nona = house_df.dropna()

# create database with for only CA homes
house_ca = house_nona[["Year", "Median_CA"]]

# Group cars by year and msrp and median.
car_msrp = cars_df_no_na[["Year", "MSRP"]].groupby("Year").median()

# reset car_msrp index
car_msrp = car_msrp.reset_index()

# merge car_msrp and house_ca dataframes on Year column
car_house = pd.merge(car_msrp, house_ca, on='Year')

# Corrilation between MSRP and Median_CA by Year
car_house[["MSRP", "Median_CA"]].corr()

,MSRP,Median_CA
MSRP,1.000000,0.883431
Median_CA,0.883431,1.000000


In [11]:
# Create Data base with difference between each year
house_diff = house_nona.set_index("DATE").diff()

# find the max difference in each state
house_diff.max()

Median_CA     3774.0
Median_CO    10590.0
Median_NY     3110.0
Median_TX     3240.0
Median_IN     7970.0
Year             1.0
dtype: float64

In [12]:
# find the date of the max difference in Colorado
house_diff[house_diff["Median_CO"] == 10590.0]["Median_CO"]

DATE
1/1/13    10590.0
Name: Median_CO, dtype: float64

In [13]:
# create dummie columns from columns with lists
def dummie(column, game_id):
    # create a new dataframe from the column series with game_id as the index
    new_df = pd.DataFrame(column.str.split(',').tolist(), index=game_id).stack()
    # reset the index
    new_df = new_df.reset_index([0, 'index'])
    # convert categorical variable into dummy variables
    new_df = pd.get_dummies(new_df)
    # sum and get rid of duplicate rows
    new_df = new_df.groupby(['index']).sum().reset_index()
    # Take 0_ out of each column name
    new_df.rename(columns=lambda x: x[2:], inplace=True)
    new_df.rename(columns={'dex': 'index'}, inplace=True)
        
    return new_df

In [14]:
# reset index in cars_df_no_na
cars_df_no_na.reset_index(level=0, inplace=True)

# Create database with dummie market categories
dummie_car_df = dummie(cars_df_no_na['Market Category'], cars_df_no_na['index'])

# Merge cars_df_no_na and dummie_car_df on index
car_cat_df = pd.merge(cars_df_no_na, dummie_car_df, on="index")

# Show count of Transmission Type by Performance count
car_cat_df.groupby("Transmission Type")["Performance"].sum()

Transmission Type
AUTOMATED_MANUAL     202.0
AUTOMATIC           1311.0
DIRECT_DRIVE           0.0
MANUAL               562.0
UNKNOWN                3.0
Name: Performance, dtype: float64